<a href="https://colab.research.google.com/github/StormStudioAndroid2/diploma_colab/blob/main/diploma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Сначала загрузим файл с партиями с Yandex Disk, обработаем данные и представим в виде матрицы

In [1]:
import requests
from urllib.parse import urlencode

def load_file(name, url):
  base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
  public_key = url  # Сюда вписываем ссылку на Yandex Disk, c нужным нам файлом

  # Получаем загрузочную ссылку
  final_url = base_url + urlencode(dict(public_key=public_key))
  response = requests.get(final_url)
  download_url = response.json()['href']

  # Загружаем файл и сохраняем его
  download_response = requests.get(download_url)
  with open(name, 'wb') as f: 
    f.write(download_response.content)


Теперь мы должны распарсить полученный нами файл. Для этого будем использовать библиотеку Pandas

In [3]:
import pandas as pd

def parse_excel(name):
  data = pd.read_excel(name, usecols ="B:J")

  votes = pd.read_excel(name, usecols ="A")
  votes = votes.values.tolist()

  parties = (pd.read_excel(name).columns.tolist()[1:])
  return data.values.tolist(), votes, parties

matrix, votes, parties = parse_excel('first.xlsx')
results = ['за', 'против', 'неучастие']


KeyError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Для того, чтобы было удобно работать в Python, преобразуем данные во вложенный список в методе parse_excel. Теперь покажем, что у нас вышло:

In [ ]:
print(matrix)

Теперь заменим значения на числа по следующему правилу: 'for' - 0, 'against' - 1, 'неучастие' - 2. Так у нас получится взвешенный граф, где ребро между партией и голосованием будет соответствовать голосу за/против/воздержался

In [ ]:
def format_matrix_to_numbers(matrix_unformatted):
  for i in range(0, len(matrix_unformatted)):
    for j in range(0, len(matrix_unformatted[0])):
      if (matrix_unformatted[i][j] == 'неучастие'):
        matrix_unformatted[i][j] = 3
      if (matrix_unformatted[i][j] == 'воздержались'):
        matrix_unformatted[i][j] = 2
      if (matrix_unformatted[i][j] == 'for'):
        matrix_unformatted[i][j] = 0
      if (matrix_unformatted[i][j] == 'against'):
        matrix_unformatted[i][j] = 1


format_matrix_to_numbers(matrix)
print(matrix)

Теперь мы будем брать по очереди ребра со значением 0, 1, 2 и делать следующее:

Оставляем ребра одного типа. Теперь проходимся по всем голосованиям, и добавляем в массив списки партий, проголосовавших одинаково в каждом голосовании. Так components состоит из списков, каждый список - это партиии, которые повели себя одинаково на каком-либо одинаково

In [ ]:
def get_components_list(matrix1, maxvalue):

  components = []

  for type_edge in range(0, maxvalue+1):
    for i in range(0, len(matrix1)):
      list_parties = []
      for j in range(0, len(matrix1[0])):
        if (matrix1[i][j]==type_edge):
          list_parties.append(j)
      components.append(list_parties)
  return components

components_all = get_components_list(matrix, 3)
print(components_all)

Теперь нам нужно построить итоговый граф, с весами, в котором ребро весом n между вершинами v и u показывает, что n раз партии u и v повели себя одинаково на голосовании

In [ ]:
def make_graph_matrix(components):

  graph_matrix = [[0] * len(parties) for _ in range(len(parties))]

  for i in range(0, len(components)):
    for first_elem in components[i]:
      for second_elem in components[i]:
        if (first_elem!=second_elem):
          graph_matrix[first_elem][second_elem]+=1
  return graph_matrix

graph_matrix_all = make_graph_matrix(components_all)

Теперь построим таблицу этих весов для наглядности, число в ячейке показывает, сколько раз эти партии одинаково вели себя на голосовании

In [ ]:
print(pd.DataFrame(graph_matrix_all, parties, parties))

Теперь нам нужно построить взвешенный граф из матрицы смежности, показать веса.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib.pyplot import figure


def scalegraph(sf):  # scale graph by scalefactor = sf, keeping it centred
    plt.axis('off')
    axis = plt.gca()
    xlim=list(axis.get_xlim())
    ylim=list(axis.get_ylim())
    xav=sum(xlim)/2
    dx=(xlim[1]-xlim[0])/2
    yav=sum(ylim)/2
    dy=(ylim[1]-ylim[0])/2
    axis.set_xlim([xav-sf*dx,xav+sf*dx])
    axis.set_ylim([yav-sf*dy,yav+sf*dy])

def plot_heatmap(matrix_heat, labels):
  fig, ax = plt.subplots()
  sns.heatmap(matrix_heat, annot=True, cmap='coolwarm')
  ax.set_xticks(np.arange(len(labels)))
  ax.set_yticks(np.arange(len(labels)))
  ax.set_xticklabels(labels)
  ax.set_yticklabels(labels)
  plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
  plt.setp(ax.get_yticklabels(), rotation=0, ha="right",
         rotation_mode="anchor")
  plt.show()

def draw_graph_matrix_adjency(matrix_adjency, votes1):
  figure(figsize=(80, 60), dpi=80)
  G = nx.Graph()
  for i in range(0, len(matrix_adjency)):
    for j in range(0, len(matrix_adjency[0])):
      if (matrix_adjency[i][j]!=0):
        G.add_edge(parties[i], parties[j], weight=matrix_adjency[i][j])
  elarge = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] >= len(votes1)/2]
  esmall = [(u, v) for (u, v, d) in G.edges(data=True) if d["weight"] < len(votes1)/2]

  pos = nx.circular_layout(G)  # positions for all nodes

  # nodes
  nx.draw_networkx_nodes(G, pos, node_size=15000, node_color='lightblue')

  # edges
  nx.draw_networkx_edges(G, pos, edgelist=elarge, width=6)
  nx.draw_networkx_edges(
      G, pos, edgelist=esmall, width=6, alpha=0.5, edge_color="b", style="dashed"
  )

  # labels
  nx.draw_networkx_labels(G, pos, font_size=35, font_family="sans-serif")
  plt.axis("off")
  scalegraph(1)
  plt.show()

def make_graph(matrix_adjency):
  G = nx.Graph()
  for i in range(0, len(matrix_adjency)):
    for j in range(0, len(matrix_adjency[0])):
      if (matrix_adjency[i][j]!=0):
        G.add_edge(parties[i], parties[j], weight=matrix_adjency[i][j])
  return G
  
figure(figsize=(80, 60), dpi=80)
draw_graph_matrix_adjency(graph_matrix_all, votes)

Теперь сделаем итоговую функцию, которая будет загружать данные по ссылке в файл и их сразу обрабатывать, выводить все нужные данные по файлу: матрицы смежности, графы

In [ ]:
import seaborn as sns # for data visualization
import matplotlib.pyplot as plt
import numpy as np

def data_parse_plotting(url, name, table_values=None):
  load_file(name, url)
  matrix, votes, parties = parse_excel(name)
  results = ['за', 'против', 'неучастие']
  format_matrix_to_numbers(matrix)
  components_all = get_components_list(matrix, 3)
  graph_matrix_all = make_graph_matrix(components_all)
  print('---Построим матрицу смежности и график с учетом, когда партии не принимали участия в голосовании')
  
  plt.figure(figsize=(20, 10))
  plot_heatmap(graph_matrix_all, parties)

  figure(figsize=(80, 60), dpi=80)
  draw_graph_matrix_adjency(graph_matrix_all, votes)
  
  print('---Построим матрицу смежности и график без учета, когда партии не принимали участия в голосовании')
  components1 = get_components_list(matrix, 2)
  graph_matrix_active = make_graph_matrix(components1)
  
  plt.figure(figsize=(20, 10))
  plot_heatmap(graph_matrix_active, parties)
  
  figure(figsize=(80, 60), dpi=80)
  draw_graph_matrix_adjency(graph_matrix_active, votes)
  if (table_values != None):
    values_list = []
    G = make_graph(graph_matrix_all)
    degree = nx.degree_centrality(G)
    betweenness = nx.betweenness_centrality(G)
    closeness = nx.closeness_centrality(G)
    values_list.append(degree)
    values_list.append(betweenness)
    values_list.append(closeness)
    values_list.append(nx.average_clustering(G))
    values_list.append(nx.density(G))
    table_values.append(values_list)

values_matrix = []
data_parse_plotting('https://disk.yandex.ru/i/uhcytFEl7xHjww', 'first.xlsx', table_values= values_matrix)

Теперь мы также должны обработать второй файл с голосованием по сельскому хозяйству:

In [ ]:
data_parse_plotting('https://disk.yandex.ru/i/MDH7Hep2Y_CdlA', 'second.xlsx', values_matrix)

Также обработаем голосования по экологии:

In [ ]:
data_parse_plotting('https://disk.yandex.ru/d/PL3IV-A8HrVDzA', 'third.xlsx', values_matrix)